In [1]:
import math
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os
%matplotlib inline

import os
import imageio
import random

from moviepy.editor import VideoFileClip
from IPython.display import HTML

## Helper Functions

In [2]:
def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
def canny(img, low_threshold, high_threshold):
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=10):
    for line in lines:
        for x1,y1,x2,y2 in line:
            #For better see the different lines in different colors
            internalColor=[random.randint(1, 255), random.randint(1, 255), random.randint(1, 255)]

            #To eliminate horizontal lines
            if(abs(y1-y2)>50):
                
                #When the lane lines are not connected to the bottom of image, 
                # calculate the bottom point of that line and make it longer to connect to bottom
                if y1!=y2 and y1!=540 and y2!=540:
                    ratio=(abs(x1-x2))/abs(y1-y2)
                    if y1>y2:                
                        if x1>x2:
                            x1=x1+(ratio*(540-y1))
                        else:
                            x1=x1-(ratio*(540-y1))
                        y1=540
                        x1=int(x1)
                    else:
                        if x1>x2:
                            x2=x2-ratio*(540-y2)
                        else:
                            x2=x2+ratio*(540-y2)
                        y2=540    
                        x2=int(x2)         
            
                cv2.line(img, (x1, y1), (x2, y2), color, thickness)
                #cv2.line(img, (x1, y1), (x2, y2), internalColor, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

def weighted_img(img, initial_img, alpha=0.8, beta=1.0, lmbd=0.0):
    return cv2.addWeighted(initial_img, alpha, img, beta, lmbd)

# Pipeline

In [3]:
def process_image(image):
    
    grayImage = grayscale(image)
    blurgrayImage = gaussian_blur(grayImage,5)
    edgeImage=canny(blurgrayImage,50,150)    
    
    imshape = image.shape
    vertices = np.array([[(50,imshape[0]),(475, 312), (505, 312), (imshape[1],imshape[0])]], dtype=np.int32)
    ROIedgeImage=region_of_interest(edgeImage,vertices)
  

    line_image=hough_lines(ROIedgeImage,1,np.pi/270, 30, 20, 120)
    lines_edges = weighted_img(image, line_image, 0.8, 1, 0)     
    
    return lines_edges

## SolidWhiteRight Video Processing

In [4]:
white_output = 'test_videos_output/solidWhiteRight.mp4'
#clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image)
%time white_clip.write_videofile(white_output, audio=False)

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

[MoviePy] >>>> Building video test_videos_output/solidWhiteRight.mp4
[MoviePy] Writing video test_videos_output/solidWhiteRight.mp4


100%|███████████████████████████████████████▊| 221/222 [00:13<00:00, 16.35it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidWhiteRight.mp4 

Wall time: 15.8 s


## Solid Yellow Left Video Processing

In [5]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'

#clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(0,5)
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

[MoviePy] >>>> Building video test_videos_output/solidYellowLeft.mp4
[MoviePy] Writing video test_videos_output/solidYellowLeft.mp4


100%|███████████████████████████████████████▉| 681/682 [00:49<00:00, 14.88it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidYellowLeft.mp4 

Wall time: 52 s
